# Cabinet Turnover in USA administrations

### Set ups

In [ ]:
## Set up the modules and libraries.

import pandas as pd
import numpy as np
import matplotlib as plotly

In [21]:
## Creating the dataframe for the tool.

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Cabinet Turnover/cabinet-turnover.csv")

display(df)

,president,position,appointee,start_date,end_date,length,departure_day,gender,Unnamed: 8,Unnamed: 9
0,Carter,OMB Director,Bert Lance,1/21/77,9/23/77,245,247.0,Male,NaN,NaN
1,Carter,Secretary of Transportation,Brock Adams,1/23/77,7/20/79,908,912.0,Male,NaN,NaN
2,Carter,"Secretary of Health, Education & Welfare",Joseph Califano Jr.,1/25/77,8/3/79,920,926.0,Male,NaN,NaN
3,Carter,Secretary of Housing & Urban Development,Patricia Harris,1/23/77,8/3/79,922,926.0,Female,NaN,NaN
4,Carter,Secretary of the Treasury,W. Michael Blumenthal,1/23/77,8/4/79,923,927.0,Male,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
374,Biden,UN Ambassador,Linda Thomas-Greenfield,2/24/21,Still in office,NaN,NaN,Female,NaN,NaN
375,Biden,CEA Chair,Cecilia Rouse,3/12/21,Still in office,NaN,NaN,Female,NaN,NaN
376,Biden,SBA Administrator,Isabel Guzman,3/17/21,Still in office,NaN,NaN,Female,NaN,NaN
377,Biden,OSTP Director,Arati Prabhakar,10/1/22,Still in office,NaN,NaN,Female,NaN,NaN


### Data Preparation

In [ ]:
## Rename the column headers

## declare the new headers in a dictionary
new_header = {"president": "President", "position":"Official Title","appointee":"Name of Member","start_date":"Start Date","end_date":"End Date","length":"Tenure (in days)","departure_day":"Departure (in days)","gender":"Gender"}

## rename the column headers using the rename function and the new headers.

df.rename(columns=new_header, inplace=True)

display(df)


In [ ]:
## Dropping empty columns

new_df = df.drop(["Unnamed: 8","Unnamed: 9"], axis=1)

display(new_df)

In [ ]:
## Create a data frame for appointees who are still in office.

still_in_office = new_df[df["End Date"] == "Still in office"]

display(still_in_office)

In [ ]:
## Filter out the rows for appointees that are still in the office.

new_df = new_df[df["End Date"] != "Still in office"]

display(new_df)

In [ ]:
## Describe the new dataframe of appointees no longer serving in the cabinet.

new_df.info()

In [ ]:
## Identify the rows with Null values for Departure (in days)

null_values = df["Departure (in days)"].isnull()

new_df[null_values]

## The tenure in days and departure in days were combined. Possibly meaning they left the administration also on their last day in office.
## We shall remove these from the data as the data types are inconsistent.

In [109]:
## There are values under the "Tenure (in days)" column that are numbers with strings.

new_df["Tenure (in days)"] = new_df["Tenure (in days)"].str.replace("combined","")
new_df["Tenure (in days)"] = new_df["Tenure (in days)"].str.replace("Approx 1613","1613")
new_df["Tenure (in days)"] = new_df["Tenure (in days)"].str.replace(". 1613","1613")

## typecase the column to numeric.

new_df["Tenure (in days)"] = new_df["Tenure (in days)"].astype(float)

<ipython-input-109-db199a5f18c2>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  new_df["Tenure (in days)"] = new_df["Tenure (in days)"].str.replace(". 1613","1613")


In [116]:
## displaying the final data frame

display(new_df)

,President,Official Title,Name of Member,Start Date,End Date,Tenure (in days),Departure (in days),Gender
0,Carter,OMB Director,Bert Lance,1/21/77,9/23/77,245.0,247.0,Male
1,Carter,Secretary of Transportation,Brock Adams,1/23/77,7/20/79,908.0,912.0,Male
2,Carter,"Secretary of Health, Education & Welfare",Joseph Califano Jr.,1/25/77,8/3/79,920.0,926.0,Male
3,Carter,Secretary of Housing & Urban Development,Patricia Harris,1/23/77,8/3/79,922.0,926.0,Female
4,Carter,Secretary of the Treasury,W. Michael Blumenthal,1/23/77,8/4/79,923.0,927.0,Male
...,...,...,...,...,...,...,...,...
349,Trump,Chief of Staff,Mark Meadows,3/31/20,1/20/21,295.0,1462.0,Male
350,Trump,Director of National Intelligence,John Ratcliffe,5/26/20,1/20/21,239.0,1462.0,Male
351,Trump,OMB Director,Russ Vought,7/22/20,1/20/21,182.0,1462.0,Male
352,Biden,OSTP Director,Eric Lander,6/2/21,2/18/22,261.0,395.0,Male


In [117]:
## Describing the numeric columns

new_df.describe()

,Tenure (in days),Departure (in days)
count,354.000000,347.000000
mean,1086.197740,1799.308357
std,627.769806,814.888641
min,42.000000,137.000000
25%,594.000000,1408.000000
50%,994.500000,1467.000000
75%,1455.000000,2921.500000
max,2922.000000,3013.000000


In [149]:
## Identifying outliers for the Tenure (in day)

q1 = new_df["Tenure (in days)"].describe()['25%']
q3 = new_df["Tenure (in days)"].describe()['75%']

# Interquartile range
iqr = q3 - q1

## getting the lower and upper fence.

lf = q1 - (1.5 * iqr)
uf = q3 + (1.5 * iqr)

print(f'Lower fence: {lf}')
print(f'Upper fence: {uf}')

Lower fence: -711.0
Upper fence: 2737.0


In [150]:
## Return data rows with Tenure (in days) that are below the lower fence or above the upper fence.

new_df[(new_df["Tenure (in days)"] < lf) & (new_df["Tenure (in days)"] > uf)]

,President,Official Title,Name of Member,Start Date,End Date,Tenure (in days),Departure (in days),Gender


In [147]:
## The outliers who are above the upper fence will be removed from the main dataframe. 

new_df = new_df[new_df["Tenure (in days)"] < uf]

### Data Exploration

In [ ]:
display(new_df)